# CSC 786 - EDR Evasion Analysis: io_uring vs Traditional Syscalls

**Author:** Michael Mendoza | **Course:** CSC 786 | **Institution:** Dakota State University

---

## Metrics Evaluated

### Primary Metrics
1. **Detection Rate** - Proportion of runs producing at least one audit hit
2. **False Negative Rate** - Runs where behavior occurred but no alert generated
3. **Time-to-Detection** - Time between execution and first alert

### Secondary Metrics
4. **io_uring Setup Detection** - Whether io_uring initialization is visible
5. **Evasion Delta** - Detection rate difference (traditional - io_uring)
6. **Path-Specific Detection** - Whether the specific test file appears in audit logs

## MITRE ATT&CK Mapping
- T1059 - Command and Scripting Interpreter (exec_cmd)
- T1071 - Application Layer Protocol (net_connect)
- T1005 - Data from Local System (file_io)
- T1562.001 - Impair Defenses (io_uring evasion)

## 0) Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('colorblind')
plt.rcParams.update({'figure.figsize': (10, 6), 'font.size': 11})

RESULTS_DIR = Path('results')
FIGURES_DIR = RESULTS_DIR / 'figures'
TABLES_DIR = RESULTS_DIR / 'tables'
for d in [FIGURES_DIR, TABLES_DIR]: d.mkdir(parents=True, exist_ok=True)
print(f'Output: {RESULTS_DIR}')

## 1) Load Data

In [ ]:
for p in [Path('../data/processed'), Path('data/processed'), Path('../../data/processed')]:
    if p.exists():
        processed_dir = p
        break
else:
    raise FileNotFoundError('No data/processed directory found')

csvs = sorted(processed_dir.glob('runs_*.csv'))
if not csvs: raise FileNotFoundError('No runs_*.csv found')

csv_path = csvs[-1]
print(f'Loading: {csv_path}')
df = pd.read_csv(csv_path)
print(f'Shape: {df.shape}')
df.head()

## 2) Preprocessing

In [ ]:
# Add missing columns for backward compatibility
if 'iouring_hits' not in df.columns: df['iouring_hits'] = 0
if 'time_to_detect' not in df.columns: df['time_to_detect'] = -1
if 'path_hits' not in df.columns: df['path_hits'] = -1

# Classify cases
df['method'] = df['case'].apply(lambda x: 'io_uring' if 'uring' in x.lower() else 'traditional')
df['operation'] = df['case'].apply(lambda x:
    'File Write' if 'file_io' in x.lower() else
    'File Read' if 'read_file' in x.lower() or 'openat' in x.lower() else
    'Network' if 'net' in x.lower() else
    'Process Exec' if 'exec' in x.lower() else 'Other')

# Derived metrics
df['total_audit_hits'] = df['file_hits'] + df['net_hits'] + df['exec_hits']
df['audit_detected'] = df['total_audit_hits'] > 0
df['wazuh_detected'] = df['wazuh_alerts'] > 0
df['iouring_detected'] = df['iouring_hits'] > 0
df['ttd_seconds'] = df['time_to_detect'].replace(-1, np.nan)
df['path_detected'] = df['path_hits'].apply(lambda x: True if x > 0 else (False if x == 0 else None))

print('Cases:', df['case'].unique().tolist())
print(f'Iterations: {df.iteration.nunique()}')

## 3) Detection Rate Analysis

In [ ]:
det_rates = df.groupby('case').agg({
    'audit_detected': 'mean',
    'iouring_detected': 'mean',
    'wazuh_detected': 'mean',
    'iteration': 'count'
}).rename(columns={'iteration': 'N', 'audit_detected': 'Audit Rate',
                   'iouring_detected': 'iouring Setup Rate', 'wazuh_detected': 'Wazuh Rate'})
det_rates['Method'] = det_rates.index.map(lambda x: 'io_uring' if 'uring' in x else 'traditional')

print('=== Detection Rates by Case ===')
display_rates = det_rates.copy()
for c in ['Audit Rate', 'iouring Setup Rate', 'Wazuh Rate']:
    display_rates[c] = (display_rates[c] * 100).round(1).astype(str) + '%'
print(display_rates.to_string())
det_rates.to_csv(TABLES_DIR / 'detection_rates.csv')

In [ ]:
method_summary = df.groupby('method')['audit_detected'].agg(['mean', 'sum', 'count'])
method_summary.columns = ['Detection Rate', 'Detections', 'Total Runs']
print('\n=== Traditional vs io_uring ===')
print(method_summary)

## 4) Path-Specific Detection (KEY EVASION METRIC)

In [ ]:
print('='*70)
print('PATH-SPECIFIC DETECTION ANALYSIS')
print('='*70)

file_ops = df[df['path_hits'] >= 0].copy()
if len(file_ops) > 0:
    path_summary = file_ops.groupby('case').agg({
        'path_hits': ['sum', 'mean'],
        'iteration': 'count'
    })
    path_summary.columns = ['Total Path Hits', 'Mean Path Hits', 'Total Runs']
    path_summary['Path Detection Rate'] = file_ops.groupby('case').apply(
        lambda x: (x['path_hits'] > 0).mean() * 100
    )
    print(path_summary.round(2).to_string())
    path_summary.to_csv(TABLES_DIR / 'path_detection.csv')
    print('\n>>> Traditional: HIGH path detection = syscalls visible')
    print('>>> io_uring: LOW/ZERO path detection = EVASION CONFIRMED')
else:
    print('No path-specific data available')

## 5) False Negative Rate

In [ ]:
print('=== False Negative Rates ===')
file_ops = df[df['path_hits'] >= 0]
if len(file_ops) > 0:
    fn_rates = file_ops.groupby('case').apply(lambda x: (x['path_hits'] == 0).mean())
else:
    fn_rates = df.groupby('case')['audit_detected'].apply(lambda x: 1 - x.mean())
fn_df = fn_rates.to_frame('False Negative Rate')
fn_df['Method'] = fn_df.index.map(lambda x: 'io_uring' if 'uring' in x else 'traditional')
fn_display = fn_df.copy()
fn_display['False Negative Rate'] = (fn_display['False Negative Rate'] * 100).round(1).astype(str) + '%'
print(fn_display.to_string())
fn_df.to_csv(TABLES_DIR / 'false_negative_rates.csv')

## 6) Time-to-Detection

In [ ]:
ttd = df[df['ttd_seconds'].notna()].groupby('case')['ttd_seconds'].agg(['count', 'median', 'mean', 'std'])
ttd.columns = ['Detected', 'Median TTD', 'Mean TTD', 'Std']
if len(ttd) > 0:
    print('=== Time-to-Detection (seconds) ===')
    print(ttd.round(3).to_string())
    ttd.to_csv(TABLES_DIR / 'time_to_detection.csv')

## 7) Syscall Bypass Validation

In [ ]:
pairs = [
    ('file_io_traditional', 'file_io_uring', 'File Write'),
    ('read_file_traditional', 'openat_uring', 'File Read'),
    ('net_connect_traditional', 'net_connect_uring', 'Network'),
]

print('='*70)
print('SYSCALL BYPASS VALIDATION')
print('='*70)

validation = []
for trad_case, uring_case, op in pairs:
    trad = df[df['case'] == trad_case]
    uring = df[df['case'] == uring_case]
    if len(trad) == 0 or len(uring) == 0: continue
    
    if 'path_hits' in df.columns and trad['path_hits'].iloc[0] >= 0:
        t_metric = (trad['path_hits'] > 0).mean() * 100
        u_metric = (uring['path_hits'] > 0).mean() * 100
    else:
        t_metric = trad['audit_detected'].mean() * 100
        u_metric = uring['audit_detected'].mean() * 100
    
    u_setup = (uring['iouring_hits'] > 0).mean() * 100
    bypass = t_metric > 50 and u_metric < 50 and u_setup > 50
    
    validation.append({'Operation': op, 'Trad Detection %': t_metric, 
                       'Uring Detection %': u_metric, 'Uring Setup %': u_setup,
                       'Bypass Confirmed': 'YES' if bypass else 'NO'})
    print(f'\n{op}: Trad={t_metric:.0f}%, Uring={u_metric:.0f}%, Setup={u_setup:.0f}%')
    print(f'  >>> Bypass: {"YES" if bypass else "NO"}')

pd.DataFrame(validation).to_csv(TABLES_DIR / 'syscall_bypass_validation.csv', index=False)

## 8) Visualizations

In [ ]:
# Figure 1: Detection Rates
fig, ax = plt.subplots(figsize=(12, 6))
rates = df.groupby('case')['audit_detected'].mean() * 100
colors = ['#2ecc71' if 'traditional' in c else '#e74c3c' for c in rates.index]
bars = ax.bar(range(len(rates)), rates.values, color=colors, edgecolor='black')
ax.set_xticks(range(len(rates)))
ax.set_xticklabels(rates.index, rotation=45, ha='right')
ax.set_ylabel('Detection Rate (%)')
ax.set_title('Figure 1: Audit Detection Rate by Test Case', fontweight='bold')
ax.set_ylim(0, 110)
for bar, val in zip(bars, rates.values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, f'{val:.0f}%', ha='center', fontweight='bold')
from matplotlib.patches import Patch
ax.legend(handles=[Patch(color='#2ecc71', label='Traditional'), Patch(color='#e74c3c', label='io_uring')], loc='lower right')
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'fig1_detection_rates.png', dpi=300)
plt.show()

In [ ]:
# Figure 2: Path Detection (Key Metric)
fig, ax = plt.subplots(figsize=(10, 6))
file_cases = df[df['path_hits'] >= 0]['case'].unique()
if len(file_cases) > 0:
    path_rates = df[df['path_hits'] >= 0].groupby('case').apply(lambda x: (x['path_hits'] > 0).mean() * 100)
    colors = ['#2ecc71' if 'traditional' in c else '#e74c3c' for c in path_rates.index]
    bars = ax.bar(range(len(path_rates)), path_rates.values, color=colors, edgecolor='black')
    ax.set_xticks(range(len(path_rates)))
    ax.set_xticklabels(path_rates.index, rotation=45, ha='right')
    ax.set_ylabel('Path Detection Rate (%)')
    ax.set_title('Figure 2: Path-Specific Detection (KEY EVASION METRIC)', fontweight='bold')
    ax.set_ylim(0, 110)
    for bar, val in zip(bars, path_rates.values):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, f'{val:.0f}%', ha='center', fontweight='bold')
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'fig2_path_detection.png', dpi=300)
plt.show()

In [ ]:
# Figure 3: Paired Comparison
fig, ax = plt.subplots(figsize=(10, 6))
labels = ['File Write', 'File Read', 'Network']
trad_cases = ['file_io_traditional', 'read_file_traditional', 'net_connect_traditional']
uring_cases = ['file_io_uring', 'openat_uring', 'net_connect_uring']
t_means = [df[df['case']==c]['total_audit_hits'].mean() for c in trad_cases]
u_means = [df[df['case']==c]['total_audit_hits'].mean() for c in uring_cases]
x = np.arange(len(labels))
ax.bar(x - 0.175, t_means, 0.35, label='Traditional', color='#2ecc71', edgecolor='black')
ax.bar(x + 0.175, u_means, 0.35, label='io_uring', color='#e74c3c', edgecolor='black')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_ylabel('Mean Audit Events')
ax.set_title('Figure 3: Mean Audit Events - Traditional vs io_uring', fontweight='bold')
ax.legend()
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'fig3_paired_comparison.png', dpi=300)
plt.show()

In [ ]:
# Figure 4: Heatmap
fig, ax = plt.subplots(figsize=(10, 6))
hm_data = df.groupby('case')[['file_hits', 'net_hits', 'exec_hits', 'iouring_hits']].apply(lambda x: (x > 0).mean() * 100)
hm_data.columns = ['File', 'Network', 'Exec', 'io_uring Setup']
hm_data = hm_data.reindex(sorted(hm_data.index, key=lambda x: (0 if 'traditional' in x else 1, x)))
sns.heatmap(hm_data, annot=True, fmt='.0f', cmap='RdYlGn', vmin=0, vmax=100, cbar_kws={'label': 'Detection %'}, ax=ax)
ax.set_title('Figure 4: Detection Heatmap', fontweight='bold')
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'fig4_heatmap.png', dpi=300)
plt.show()

In [ ]:
# Figure 5: Evasion Effectiveness
fig, ax = plt.subplots(figsize=(8, 5))
ops = ['File Write', 'File Read', 'Network']
evasion = []
for t, u in [('file_io_traditional', 'file_io_uring'), ('read_file_traditional', 'openat_uring'), ('net_connect_traditional', 'net_connect_uring')]:
    trad_df = df[df['case']==t]
    uring_df = df[df['case']==u]
    if 'path_hits' in df.columns and trad_df['path_hits'].iloc[0] >= 0:
        tr = (trad_df['path_hits'] > 0).mean() * 100
        ur = (uring_df['path_hits'] > 0).mean() * 100
    else:
        tr = trad_df['audit_detected'].mean() * 100
        ur = uring_df['audit_detected'].mean() * 100
    evasion.append(tr - ur)
colors = ['#3498db' if e > 50 else '#f39c12' if e > 0 else '#95a5a6' for e in evasion]
ax.barh(ops, evasion, color=colors, edgecolor='black')
for i, v in enumerate(evasion): ax.text(max(v + 2, 5), i, f'{v:.0f}%', va='center', fontweight='bold')
ax.set_xlabel('Detection Rate Reduction (%)')
ax.set_title('Figure 5: io_uring Evasion Effectiveness', fontweight='bold')
ax.axvline(0, color='black', linewidth=0.5)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'fig5_evasion.png', dpi=300)
plt.show()

## 9) MITRE ATT&CK Mapping

In [ ]:
mitre = pd.DataFrame([
    {'Technique': 'T1059', 'Name': 'Command and Scripting Interpreter', 'Test Case': 'exec_cmd_traditional', 'io_uring Evasion': 'N/A (execve not supported)'},
    {'Technique': 'T1005', 'Name': 'Data from Local System', 'Test Case': 'file_io_*, read_file_*, openat_*', 'io_uring Evasion': 'Yes'},
    {'Technique': 'T1071', 'Name': 'Application Layer Protocol', 'Test Case': 'net_connect_*', 'io_uring Evasion': 'Yes'},
    {'Technique': 'T1562.001', 'Name': 'Impair Defenses: Disable/Modify Tools', 'Test Case': 'All io_uring variants', 'io_uring Evasion': 'Primary finding'},
])
print('=== MITRE ATT&CK Mapping ===')
print(mitre.to_string(index=False))
mitre.to_csv(TABLES_DIR / 'mitre_mapping.csv', index=False)

## 10) Executive Summary

In [ ]:
print('='*70)
print('EXECUTIVE SUMMARY')
print('='*70)
print(f'Total runs: {len(df)}')
print(f'Iterations: {df.iteration.nunique()}')

trad_df = df[df['method'] == 'traditional']
uring_df = df[df['method'] == 'io_uring']

print(f'\nTraditional detection rate: {trad_df.audit_detected.mean():.1%}')
print(f'io_uring detection rate: {uring_df.audit_detected.mean():.1%}')

file_trad = df[(df['method'] == 'traditional') & (df['path_hits'] >= 0)]
file_uring = df[(df['method'] == 'io_uring') & (df['path_hits'] >= 0)]
if len(file_trad) > 0 and len(file_uring) > 0:
    trad_path = (file_trad['path_hits'] > 0).mean()
    uring_path = (file_uring['path_hits'] > 0).mean()
    print(f'\nPath-specific detection (Traditional): {trad_path:.1%}')
    print(f'Path-specific detection (io_uring): {uring_path:.1%}')
    print(f'\n>>> EVASION EFFECTIVENESS: {(trad_path - uring_path)*100:.1f}%')

print(f'\nio_uring setup detection: {uring_df.iouring_detected.mean():.1%}')
print('\nConclusion: io_uring bypasses syscall monitoring.')
print('Defenders can detect setup but not operations.')
print('='*70)
print(f'\nFigures: {list(FIGURES_DIR.glob("*.png"))}')
print(f'Tables: {list(TABLES_DIR.glob("*.csv"))}')